In [22]:
%matplotlib inline
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.contrib.layers import flatten

train_data = pd.read_csv("./fashion-mnist_train.csv")
train_labels = train_data.iloc[:, 0]
train_pixels = train_data.iloc[:,1:785]
train_pixels = train_pixels.values

X_train, y_train = train_pixels[0:50000,:], train_labels[0:50000]
X_val, y_val = train_pixels[50000:60000,:], train_labels[50000:60000]

test_data = pd.read_csv("./fashion-mnist_test.csv")
y_test = test_data.iloc[:, 0]
X_test = test_data.iloc[:,1:785]
X_test = X_test.values

n_inputs = 28*28
n_hidden1 = 50
n_hidden2 = 50
n_outputs = 10


X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int64, shape = (None), name = "y")
hidden1 = fully_connected(X, n_hidden1, activation_fn = tf.nn.relu)
hidden2 = fully_connected(hidden1, n_hidden2, activation_fn = tf.nn.relu)

with tf.name_scope("output"):
    logits = tf.layers.dense(hidden2, n_outputs)
    Y_proba = tf.nn.softmax(logits)

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

n_epochs = 100
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(50000 // batch_size):
            X_batch, y_batch = X_train[(batch_size*iteration):(batch_size*(iteration+1)),:], y_train[(batch_size*iteration):(batch_size*(iteration+1))]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch%10 ==0:
            acc_val = accuracy.eval(feed_dict={X: X_val,y: y_val})
            print("Epoch",epoch,":", acc_val)
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final accuracy on test set:", acc_test)

Epoch 0 : 0.7736
Epoch 10 : 0.8389
Epoch 20 : 0.8451
Epoch 30 : 0.8431
Epoch 40 : 0.8466
Epoch 50 : 0.8576
Epoch 60 : 0.8622
Epoch 70 : 0.8651
Epoch 80 : 0.865
Epoch 90 : 0.863
Epoch 100 : 0.8644
Epoch 110 : 0.8639
Epoch 120 : 0.8608
Epoch 130 : 0.8653
Epoch 140 : 0.8627
Epoch 150 : 0.8642
Epoch 160 : 0.86
Epoch 170 : 0.8591
Epoch 180 : 0.8627
Epoch 190 : 0.8621
Final accuracy on test set: 0.8555
